In [1]:
import numpy as np
import os
import pandas as pd

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import seaborn as sns

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dropout, InputLayer, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

import warnings
warnings.filterwarnings(action='ignore')

#한글설정
import matplotlib.font_manager as fm

font_dirs = ['/usr/share/fonts/truetype/nanum', ]
font_files = fm.findSystemFonts(fontpaths=font_dirs)

for font_file in font_files:
    fm.fontManager.addfont(font_file)
    
# 한글 출력을 위해서 폰트 옵션을 설정합니다.
# "axes.unicode_minus" : 마이너스가 깨질 것을 방지

sns.set(font="NanumBarunGothic",
        rc={"axes.unicode_minus":False},
        style='darkgrid')

#GPU 사용 설정, -1이면 CPU 사용
# os.environ["CUDA_VISIBLE_DEVICES"] = "0, 1"

gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
    try:
        for i in range(len(gpus)):
            tf.config.experimental.set_memory_growth(gpus[i], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


## 결과 2) method1_entropy

- **LSH ⇒ 최종 모델 정리 (val_acc, crossvalidation)**
- method1_entropy에 대한 정성적 비교..
- method1_entropy : ( E(0to1) - E(1to0) ) * entropy
- top10에 대해서 조사한 내용 표로 만들기

**A4용지 절반 나눠서 위에 사망_표, 사망_absum / 생존_표, 생존_absum 제시**

- **guri, kelly ⇒ 표 내용, 스토리텔링**
- PRE, PRO, LAB 나눌 수는 있지만.. 복잡함 (합치는 게 나을 듯)
- 사망marker, 생존marker에 대한 top5를 각각 고르는 방안 고려...

**absum 그래프 그림**

# Method1 - all fit 사용 + lambda
일관성, 방향성 고려한 feature 탐지 <br>
**goal : 사망/생존 마커 탐지** <br>
- Score 계산 식

  - <div> $Score = {E(0to1) - E(1to0)} * Entropy^{ λ}$ </div>

- $E(0to1), E(1to0)$
  1. X의 각 feature의 값(4068개)을 모두 1, 0로 바꾸어 예측
  2. 예측값 리스트의 평균값 계산
  3. 1로 바꾼 E(0to1)에서 0으로 바꾼 E(0to1)을 뺀다.
  
- `Entropy([proportion of 1, proportion of 0])^{λ}` 를 곱해준다.
  - λ (0 or 1) 적용 (0이면 entropy X, 1이면 entropy O)

In [12]:
path = '/project/LSH/'
model_path = path + 'model/allfit_ep500/allfit_ep500_seed42-17-0.7619.hdf5'

In [9]:
COLS = list(pd.read_csv(path + 'total_data_7727.csv')['ITEMID'].sort_values().unique())
x = np.load(path + 'x_(7727,10,4068).npy')
y = np.load(path + 'y_(7727,1).npy')

In [10]:
def entropy(ratio_list):
    one_ratio, zero_ratio = ratio_list[0], ratio_list[1] 
    return - ((one_ratio * (np.log2(one_ratio))) + (zero_ratio * (np.log2(zero_ratio))))

X = np.load('/project/LSH/x_(7727,10,4068).npy')

entropy_dict = {}
for i in tqdm(range(len(COLS))):
    one_ratio = X[:,:,i].sum() / (X.shape[0]*X.shape[1])
    zero_ratio = 1 - one_ratio
    entropy_num = entropy([one_ratio, zero_ratio])
    entropy_dict[COLS[i]] = entropy_num

  0%|          | 0/4068 [00:00<?, ?it/s]

In [ ]:
with tf.device('/device:GPU:1'): 
    model = tf.keras.models.load_model(model_path)
    result = []
    for i in tqdm(range(X.shape[2])):
        save_cols = X[:,:,i].copy()
        #-----zero2one-----
        X[:,:,i] = 1
        pred1 = model.predict(X, batch_size=10000, workers=-1, use_multiprocessing=True)
        mean_pred1 = np.mean(pred1)
        #-----one2zero-----
        X[:,:,i] = 0
        pred2 = model.predict(X, batch_size=10000, workers=-1, use_multiprocessing=True)
        mean_pred2 = np.mean(pred2)

        result.append({'feature' : str(COLS[i]), 'zero2one' : mean_pred1, 'one2zero' : mean_pred2,
                       'lambda0' : mean_pred1 - mean_pred2, 'lambda1' : (mean_pred1 - mean_pred2) * entropy_dict[COLS[i]]})
        #값 복원
        X[:,:,i] = save_cols

  0%|          | 0/4068 [00:00<?, ?it/s]

In [ ]:
df = pd.DataFrame(result)
df.to_csv('./Result2,3.csv', index=False)
df.sort_values('lambda1', ascending=False)